# Getting started with tomsup

Tomsup, Theory of Mind Simulation using Python, is a Python Package for Agent Based simulations. It includes 1) a framework for running agent based simulations using 2 by 2 payoffmatrices and most notably 2) an implementation of game theory of mind in a agent based framework following the implementation of [Devaine, et al. (2017)](http://dx.plos.org/10.1371/journal.pcbi.1005833).

This tutorial will simply introduce the framework of tomsup, for an introduction to the theory of mind (ToM) agent, see *introduction_to_tom.ipynb*.

It is also possible for create your own agents for an introduction on this see *creating_an_agent.ipynb*.


In [1]:
#assuming you are in the github folder change the path - not relevant if tomsup is installed via. pip
import os
os.chdir("..") # go out of the tutorials folder

In [2]:
import tomsup as ts

To get an overview of possible 

In [3]:
ts.valid_agents()

{'RB': {'name': 'Random Bias',
  'shorthand': 'RB',
  'example': 'RB(bias = 0.7)',
  'reference': 'Devaine, et al. (2017)',
  'strategy': 'Chooses 1 randomly based on a probability or bias'},
 'WSLS': {'name': 'Win-stay, lose-switch',
  'shorthand': 'WSLS',
  'example': 'WSLS()',
  'reference': 'Nowak & Sigmund (1993)',
  'strategy': 'If it win it chooses the same option again, if it lose it change to another'},
 'TFT': {'name': 'Tit-for-Tat',
  'shorthand': 'TFT',
  'example': 'TFT()',
  'reference': 'Shelling (1981)',
  'strategy': 'Intended the prisoners dilemma. It starts out cooperating and then simply copies it opponents action.'},
 'QL': {'name': 'Q-Learning Model',
  'shorthand': 'QL',
  'example': 'QL(learning_rate = 0.5, b_temp = 1)',
  'reference': 'Watkinns (1992)',
  'strategy': 'A simple reinforcement learning model, which is more choose e.g. 1 if 1 have previously been shown to yield positive result.'},
 'TOM': {'name': 'Theory of Mind',
  'shorthand': 'TOM',
  'example'

---
## 1) Creating an agent
First we will set up a Random bias (RB) agent. This agent simply choses randomly with a given bias.
There is two ways to setup an agent, either using the agent class (e.g. RB) or using the ```create_agents()``` function. We will start by calling the agent class RB directy. For a whole list of valid agent use the ```ts.valid_agents()```.

In [4]:
jung = ts.RB(bias = 0.8, save_history = True) #calling the agent subclass RB - for more on save_history see '3) inspecting Agent and AgentGroup'

# Let's examine the charlie
print(f"jung is an class of type: {type(jung)}")
if isinstance(jung, ts.Agent):
    print(f"but jung is also of has the parent class ts.Agent")

# let us have charlie make a choice 
choice = jung.compete()

print(f"jung chose {choice} and his probability for choosing 1 was {jung.get_bias()}.")

jung is an class of type: <class 'tomsup.agent.RB'>
but jung is also of has the parent class ts.Agent
jung chose 1 and his probability for choosing 1 was 0.8.


As previously mentioned you can also create agents using the create_agent function. Here we will create skinner as a Q-learning agent, which is a simple reinforcement learning agent, see Watkinns (1992) for more.

In [5]:
skinner = ts.create_agents(agents = "QL", start_params = {'save_history': True}) # create a reinforcement learning agent

Since skinner is a reinforcement learning agent his compete function requires him to know which game he is playing, so he can choose based on payoff. He also needs to know his opponents move during their last turn, so that he can update his belief about his opponent choices.

Let us have jung and skinner play the the matching pennies game. We can fetch the game from the function `PayoffMatrix`,

In [6]:
penny = ts.PayoffMatrix(name = "penny_competitive") # fetch the competitive matching pennies game.

#print the payoff matrix
print(penny)

#fetch the underlying numpy matrix
print(penny.get_matrix())

<Class PayoffMatrix, Name = penny_competitive> 
The payoff matrix of agent 0
       |  Choice agent 1
       |  - |  0 |  1 |
Choice |  0 |  1 | -1 |
agent 0|  1 | -1 |  1 |
-----------------------
The payoff matrix of agent 1
       |  Choice agent 1
       |  - |  0 |  1 |
Choice |  0 | -1 |  1 |
agent 0|  1 |  1 | -1 |
-----------------------
[[[ 1 -1]
  [-1  1]]

 [[-1  1]
  [ 1 -1]]]


Let us try to have skinner and jung compete in the matching pennies game:

In [7]:
jung_a = jung.compete() # a for action
skinner_a = skinner.compete(p_matrix = penny, agent = 1, op_choice = None) #Note that op_choice can be unspecified (or None) in the first round

jung_p = penny.payoff(action_agent0 = jung_a, action_agent1 = skinner_a, agent = 0)
skinner_p = penny.payoff(action_agent0 = jung_a, action_agent1 = skinner_a, agent = 1)

print(f"jung chose {jung_a} and skinner chose {skinner_a}, which results in a payoff for jung of {jung_p} and skinner of {skinner_p}.")
# Note that you might get different results simply by chance

jung chose 1 and skinner chose 1, which results in a payoff for jung of 1 and skinner of -1.


---
## 2) Running a tournament
In the above case we saw how to have two agents compete for a single round. It is however rare that we only need 1 round and while the above functionality can be wrapped within a for we have made a `compete()` function for convenience. In this section we will also examine the class `AgentGroup`, which allows you to run tournaments with multiple agents.

Let us start with having the two agent compete for 30 rounds in the matching pennies game:

In [8]:
results = ts.compete(jung, skinner, p_matrix = penny, n_rounds = 50)
print(type(results))

jung_sum = results['payoff_agent0'].sum()
skinner_sum = results['payoff_agent1'].sum()

print(f"jung seemed to get a total of {jung_sum} points, while skinner got a total of {skinner_sum}.")

results.head() #inspect the first 5 rows of the df

<class 'pandas.core.frame.DataFrame'>
jung seemed to get a total of -30 points, while skinner got a total of 30.


,round,choice_agent0,choice_agent1,payoff_agent0,payoff_agent1
0,0,1,1,1,-1
1,1,1,0,-1,1
2,2,1,0,-1,1
3,3,0,0,1,-1
4,4,0,0,1,-1


We see that the output of the compete function if a pandas dataframe. It is possible to change this to a list by specifying `return_val = "list"`, but having it as a dataframe allow for convenient operators attributes such as mean() and sum().

The above case a the simplest possible version of the `compete()` function. You can also specify number of simulations, whether the agent should be reset after each simulation (this is recommended) and whether it should print what simulation it is running (`silent`).




In [9]:
results = ts.compete(jung, skinner, penny, n_rounds = 50, n_sim = 10, reset_agent = True, return_val = 'df', silent = False)
results.head()

Running simulation 1 out of 10
	Running simulation 2 out of 10
	Running simulation 3 out of 10
	Running simulation 4 out of 10
	Running simulation 5 out of 10
	Running simulation 6 out of 10
	Running simulation 7 out of 10
	Running simulation 8 out of 10
	Running simulation 9 out of 10
	Running simulation 10 out of 10


,n_sim,round,choice_agent0,choice_agent1,payoff_agent0,payoff_agent1
0,0,0,1,0,-1,1
1,0,1,1,0,-1,1
2,0,2,1,0,-1,1
3,0,3,1,0,-1,1
4,0,4,1,0,-1,1


In [10]:
results.groupby('n_sim')['payoff_agent0'].sum()



n_sim
0   -30
1   -26
2   -24
3   -30
4   -30
5   -32
6   -38
7   -32
8   -42
9   -34
Name: payoff_agent0, dtype: int64

**Note** that by adding simulations the dataframe now also have a column called 'n_sim', for which simulations in which the results belongs.

### AgentGroup
Now as promised, let us take a look at tournaments with multiple agents. We will start of by creating a group of agents using a list of the desired agents as well as a list of their starting parameters. If you are in doubt how to specify these you can always the starting parameters from an existing agent using `jung.get_start_params()`.

In [20]:
agents = ['RB', 'WSLS', 'QL', '0-tom', '1-tom', '2-tom'] # create a list of agents
start_params = [{'bias': 0.8}, {}, {}, {}, {}, {}] # create a list of their starting parameters (an empty dictionary {} simply assumes defaults)


group = ts.AgentGroup(agents, start_params) # create a group of agents
print(group._agents)
print("\n----\n") # to space out the outputs

group.set_env(env = 'round_robin') # round_robin e.g. each agent will play against all other agents

group.pairing
group._agents
# make them compete
results = group.compete(p_matrix = penny, n_rounds = 100, n_sim = 5)
results.head() #examine the first 5 rows in results

{'RB_0': <tomsup.agent.RB object at 0x123667a58>, '1-tom_0': <tomsup.agent.TOM object at 0x123667a90>, 'WSLS_0': <tomsup.agent.WSLS object at 0x123667b00>, '2-tom_0': <tomsup.agent.TOM object at 0x123667b38>, 'QL_0': <tomsup.agent.QL object at 0x123667ba8>, '0-tom_0': <tomsup.agent.TOM object at 0x123667be0>}

----

Currently the pair, ('RB_0', '1-tom_0'), is competing for 5 simulations, each containg 100 rounds.
	Running simulation 1 out of 5
	Running simulation 2 out of 5
	Running simulation 3 out of 5
	Running simulation 4 out of 5
	Running simulation 5 out of 5
Currently the pair, ('RB_0', 'WSLS_0'), is competing for 5 simulations, each containg 100 rounds.
	Running simulation 1 out of 5
	Running simulation 2 out of 5
	Running simulation 3 out of 5
	Running simulation 4 out of 5
	Running simulation 5 out of 5
Currently the pair, ('RB_0', '2-tom_0'), is competing for 5 simulations, each containg 100 rounds.
	Running simulation 1 out of 5
	Running simulation 2 out of 5
	Running simul

,n_sim,round,choice_agent0,choice_agent1,payoff_agent0,payoff_agent1,agent0,agent1
0,0,0,0,0,1,-1,RB_0,1-tom_0
1,0,1,1,1,1,-1,RB_0,1-tom_0
2,0,2,1,0,-1,1,RB_0,1-tom_0
3,0,3,1,1,1,-1,RB_0,1-tom_0
4,0,4,1,0,-1,1,RB_0,1-tom_0


In [24]:
results['pair'] = results[['agent0', 'agent1']].apply(lambda x: ' vs '.join(x), axis=1)
results.groupby(['pair', 'n_sim', 'agent0'])['payoff_agent0'].sum()

pair                n_sim  agent0 
1-tom_0 vs 0-tom_0  0      1-tom_0     12
                    1      1-tom_0     20
                    2      1-tom_0      0
                    3      1-tom_0     20
                    4      1-tom_0      4
1-tom_0 vs 2-tom_0  0      1-tom_0    -18
                    1      1-tom_0      6
                    2      1-tom_0     -4
                    3      1-tom_0      2
                    4      1-tom_0     -6
1-tom_0 vs QL_0     0      1-tom_0     92
                    1      1-tom_0     90
                    2      1-tom_0     94
                    3      1-tom_0     98
                    4      1-tom_0     96
1-tom_0 vs WSLS_0   0      1-tom_0     98
                    1      1-tom_0     96
                    2      1-tom_0     90
                    3      1-tom_0     94
                    4      1-tom_0     94
2-tom_0 vs 0-tom_0  0      2-tom_0     12
                    1      2-tom_0      6
                    2      2-tom_0     14

In [74]:
agent_names = [agent + '_' + str(idx) for agent in set(agents) for idx in range(agents.count(agent))]
agent_names

['2-tom_0', '0-tom_0', 'QL_0', '1-tom_0', 'RB_0']

In [83]:
list(zip(agent_names, agents, start_params))
#_agents = {name: Agent(agent, **param) for name, agent, param in zip(agent_names, agents, start_params)}
print(agents)
print(agent_names)

['RB', 'QL', '0-tom', '1-tom', '2-tom']
['2-tom_0', '0-tom_0', 'QL_0', '1-tom_0', 'RB_0']


In [81]:
{name: ts.Agent(name.split('_')[0], **param) for name, param in zip(agent_names, start_params)}


{'2-tom_0': <tomsup.agent.TOM at 0x12c5b8748>,
 '0-tom_0': <tomsup.agent.TOM at 0x12c5b86a0>,
 'QL_0': <tomsup.agent.QL at 0x12cf24b00>,
 '1-tom_0': <tomsup.agent.TOM at 0x12cf24e48>,
 'RB_0': <tomsup.agent.RB at 0x12cf24f28>}

As you can see once the group is created and environment it is easy to have the agent compete with one another.

(for more possible environment, see `help(group.set_env)`)

---
## 3) Inspecting Agent and AgentGroup
So let's examine some of the attributes of the agents, which applies to all agents. In this section we will also look a bit into how to extract an agent from an agentGroup and how to examine the enviroment.

In [11]:
# What if I want to know the starting parameters?
print("This is the starting parameters of jung: ",    jung.get_start_params()) # Note that it also prints out default parameters
print("This is the starting parameters of skinner: ", skinner.get_start_params())

# What if I want to know the agent last choice?
print("This is jung's last choice: ",    jung.get_choice())
print("This is skinner's last choice: ", skinner.get_choice())

# What if I want to know the agents strategy?
print("jung's strategy is: ", jung.get_strategy())
print("skinner's strategy is: ", skinner.get_strategy())

This is the starting parameters of jung:  {'bias': 0.7, 'var': 0, 'save_history': True}
This is the starting parameters of skinner:  {'learning_rate': 0.5, 'b_temp': 0.001, 'expec_val': [0.5, 0.25], 'save_history': True}
This is jung's last choice:  0
This is skinner's last choice:  0
jung's strategy is:  RB
skinner's strategy is:  QL


We can also get the history, recall that we specified `get_history = True` for skinner (and jung). This means we can go back and see all his previous state for the Random bias (RB) this only include the choice. This is by default returned as a dataframe. Note that by default `get_history` is `False` to save memory.

In [12]:
# What is the history of skinner (e.g. what is his choices and internal states)

history = jung.get_history(format = "df")
print(history.head())

print("\n --- \n") # for spacing

history = skinner.get_history(format = "df")
print(history.head(15)) # the first 15 rows

choice
0       1
1       1
2       1
3       1
4       0

 --- 

    choice  expected_value0  expected_value1
0      0.0              0.5             0.25
1      0.0              0.5             0.25
2      0.0              0.5             0.25
3      0.0              0.5             0.25
4      0.0              0.5             0.25
5      0.0              0.5             0.25
6      0.0              0.5             0.25
7      0.0              0.5             0.25
8      0.0              0.5             0.25
9      0.0              0.5             0.25
10     0.0              0.5             0.25
11     0.0              0.5             0.25
12     0.0              0.5             0.25
13     0.0              0.5             0.25
14     0.0              0.5             0.25


In the above we can see the history of the two agents, both their internal states, which is none for the the random bias agent (RB) and includes two for the reinforcement learning agent (QL). The two for for the RL indicates the expected value of choosing 0 rather than 1. As expected we here see that the expected value of choosing 0 increases due to the jungs bias of 0.7.

**Note** that if the agent are competing using multiple simulations it will resets the agent after each simulations, consequently their history is also reset.